In [1]:
!pip install pyLDAvis

  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97716 sha256=c865efc7494ea520cc24d62028145dbb0fca5c25f229cbf5a50fa500ed551722
  Stored in directory: c:\users\myhome\appdata\local\pip\cache\wheels\3b\fb\41\e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
Successfully built pyLDAvis


# 전처리

In [49]:
import pandas as pd
import re
from datetime import datetime

data = pd.read_csv("hanwha.csv", index_col=None)

In [50]:
df = data.dropna()

# df = df.drop_duplicates(['date'])
df["date"] = df.date.str.replace("\. .*", "").str.replace("\.","-")
df["date"] = df.date.astype('datetime64[ns]')
# df = df.set_index("date")
# df = df.sort_index()
df = df.sort_values(by=['date'], axis=0)
df = df.reset_index().drop(["index"], axis=1)
df

&lt;&gt;:4: DeprecationWarning: invalid escape sequence \.
&lt;&gt;:4: DeprecationWarning: invalid escape sequence \.
&lt;&gt;:4: DeprecationWarning: invalid escape sequence \.
&lt;&gt;:4: DeprecationWarning: invalid escape sequence \.
&lt;&gt;:4: DeprecationWarning: invalid escape sequence \.
&lt;&gt;:4: DeprecationWarning: invalid escape sequence \.
&lt;ipython-input-50-9ed65366409c&gt;:4: DeprecationWarning: invalid escape sequence \.
  df[&quot;date&quot;] = df.date.str.replace(&quot;\. .*&quot;, &quot;&quot;).str.replace(&quot;\.&quot;,&quot;-&quot;)
&lt;ipython-input-50-9ed65366409c&gt;:4: DeprecationWarning: invalid escape sequence \.
  df[&quot;date&quot;] = df.date.str.replace(&quot;\. .*&quot;, &quot;&quot;).str.replace(&quot;\.&quot;,&quot;-&quot;)


,content,date,media,title,url
0,[이데일리 정수영 기자] 을미년 새해 초부 건설업계에 칼바람이 불고 있다. 시공능력...,2015-01-01,이데일리,연초부터 건설업계 '칼바람'..새해도 악재 '산 넘어 산',https://news.naver.com/main/read.nhn?mode=LSD&...
1,[이데일리 성문재 기자] 중화학 업계는 그야말로 다사다난했던 2014년을 보냈다. ...,2015-01-01,이데일리,"[새해전망]④중화학, 2015년에도 '도찐개찐'",https://news.naver.com/main/read.nhn?mode=LSD&...
2,상반기 기준금리 인하시 추가하락 불가피 (서울=연합뉴스) 이지헌 홍국기 기자 ...,2015-01-01,연합뉴스,은행권 연말연초에 예금금리 또 인하,https://news.naver.com/main/read.nhn?mode=LSD&...
3,[이데일리 정태선 기자] 새롭게 조직을 정비한 전자업계는 올해 ‘안정 속 성장’으로...,2015-01-01,이데일리,[비즈니스 X파일]그룹내 특공대는 어디,https://news.naver.com/main/read.nhn?mode=LSD&...
4,◇지역단장 ▲동부광진 나주호 ▲충북 송정섭 ▲충남 임장혁 ▲무등 박희창 ▲포항...,2015-01-01,연합뉴스,<인사> 한화생명,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
34942,[이데일리 경계영 기자] ◇대표이사 내정△㈜한화(000880) 글로벌부문 김맹윤 △...,2020-09-28,이데일리,[인사]한화그룹 대표이사,https://news.naver.com/main/read.nhn?mode=LSD&...
34943,[이데일리 이광수 기자] 흥국증권은 한화솔루션(009830)에 대해 미국 수소차 업...,2020-09-28,이데일리,"""한화솔루션, 가장 싼 뉴딜주…니콜라 이슈 과하게 반영""",https://news.naver.com/main/read.nhn?mode=LSD&...
34944,[이데일리 김정유 기자] 한화그룹은 한화솔루션 전략부문에 김동관 부사장(사진)을 사...,2020-09-28,이데일리,[프로필]김동관 한화솔루션 전략부문 대표이사,https://news.naver.com/main/read.nhn?mode=LSD&...
34945,(서울=연합뉴스) 28일 한화그룹 사장단 인사에서 김승연 한화그룹 회장의 장남인 김...,2020-09-28,연합뉴스,"한화, 김동관 한화솔루션 대표이사 사장 승진",https://news.naver.com/main/read.nhn?mode=LSD&...


In [51]:
df["content"] = df["content"].str.replace("\[.+기자\]", "").str.strip()
df["content"] = df["content"].str.replace("[가-힝]{3} \([a-zA-Z0-9]+@[a-zA-Z0-9]+\.co\.kr\)", "").str.strip()
df["content"] = df["content"].str.replace("\([a-zA-Z0-9]+@[a-zA-Z0-9]+\.co\.kr\)", "").str.strip()
df["content"] = df["content"].str.replace("\(.+=연합뉴스\).+기자 =", "").str.strip()
df["content"] = df["content"].str.replace("\(연합뉴스\)", "").str.strip()
df["content"] = df["content"].str.replace("\([가-힣]+=연합뉴스\)", "").str.strip()
df["content"] = df["content"].str.replace("\([가-힣]+ 연합뉴스\)", "").str.strip()
df["content"] = df["content"].str.replace("[a-zA-Z0-9]+@[a-zA-Z0-9]+\.co\.kr", "").str.strip()
df["content"] = df["content"].str.replace("\<저작권자.+금지\>", "").str.strip()
df["content"] = df["content"].str.replace("▲", "").str.strip()
df["content"] = df["content"].str.replace("◇", "").str.strip()
df["content"] = df["content"].str.replace("◆", "").str.strip()
df["content"] = df["content"].str.replace("▷", "").str.strip()
df["content"] = df["content"].str.replace("△", "").str.strip()
df["content"] = df["content"].str.replace("\[이데일리.+\]", "").str.strip()
df["content"] = df["content"].str.replace("\[.+특파원\]", "").str.strip()
df["content"] = df["content"].str.replace("■", "").str.strip()
df["content"] = df["content"].str.replace("○", " ").str.strip()
df["content"] = df["content"].str.replace("ⓝ", " ").str.strip()
df["content"] = df["content"].str.replace("<<.+>>", "").str.strip()
df["content"] = df["content"].str.replace("[0-9]{4}\.[0-9]+\.[0-9]+", "").str.strip()
df["content"] = df["content"].str.replace("문의는.*홈페이지.*\.", "").str.strip()
df["content"] = df["content"].str.replace("☎", "").str.strip()
df["content"] = df["content"].str.replace("[0-9]+-[0-9]+-[0-9]+", "").str.strip()
df["content"] = df["content"].str.replace("www\.[a-zA-Z0-9]+\.co\.kr", "").str.strip()
df["content"] = df["content"].str.replace("\(대표이사.+\)", "").str.strip()
df["content"] = df["content"].str.replace("\\'", "").str.strip()
df["content"] = df["content"].str.replace("\'", "").str.strip()
df["content"] = df["content"].str.replace("\"", "").str.strip()
df["content"] = df["content"].str.replace("‘", "").str.strip()
df["content"] = df["content"].str.replace("’", "").str.strip()
df["content"] = df["content"].str.replace("“", "").str.strip()
df["content"] = df["content"].str.replace("”", "").str.strip()
df["content"] = df["content"].str.replace("=", " ").str.strip()
df["content"] = df["content"].str.replace("--", " ").str.strip()
df["content"] = df["content"].str.replace("http:\/\/www\.[a-zA-Z0-9]+\.com\/", "").str.strip()
df["content"] = df["content"].str.replace("https:\/\/www\.[a-zA-Z0-9]+\.com\/", "").str.strip()
df["content"] = df["content"].str.replace("http:\/\/www\.[a-zA-Z0-9]+\.com", "").str.strip()
df["content"] = df["content"].str.replace("https:\/\/www\.[a-zA-Z0-9]+\.com", "").str.strip()
df["content"] = df["content"].str.replace("www\.[a-zA-Z0-9]+\.com\/[a-zA-Z0-9]+", "").str.strip()
df["content"] = df["content"].str.replace("www\.[a-zA-Z0-9]+\.com", "").str.strip()
df["content"] = df["content"].str.replace("추가 정보:", " ").str.strip()
df["content"] = df["content"].str.replace("자료 제공:", " ").str.strip()
df["content"] = df["content"].str.replace("\[.+금지\]", "").str.strip()
df["content"] = df["content"].str.replace("\(\)", "").str.strip()
df["content"] = df["content"].str.replace("[ ]+", " ").str.strip()

&lt;&gt;:1: DeprecationWarning: invalid escape sequence \[
&lt;&gt;:2: DeprecationWarning: invalid escape sequence \(
&lt;&gt;:3: DeprecationWarning: invalid escape sequence \(
&lt;&gt;:4: DeprecationWarning: invalid escape sequence \(
&lt;&gt;:5: DeprecationWarning: invalid escape sequence \(
&lt;&gt;:6: DeprecationWarning: invalid escape sequence \(
&lt;&gt;:7: DeprecationWarning: invalid escape sequence \(
&lt;&gt;:8: DeprecationWarning: invalid escape sequence \.
&lt;&gt;:9: DeprecationWarning: invalid escape sequence \&lt;
&lt;&gt;:15: DeprecationWarning: invalid escape sequence \[
&lt;&gt;:16: DeprecationWarning: invalid escape sequence \[
&lt;&gt;:21: DeprecationWarning: invalid escape sequence \.
&lt;&gt;:22: DeprecationWarning: invalid escape sequence \.
&lt;&gt;:25: DeprecationWarning: invalid escape sequence \.
&lt;&gt;:26: DeprecationWarning: invalid escape sequence \(
&lt;&gt;:36: DeprecationWarning: invalid escape sequence \/
&lt;&gt;:37: DeprecationWarning: invalid escap

In [52]:
# df.iloc[31599].content

df.content[0]

&#39;을미년 새해 초부 건설업계에 칼바람이 불고 있다. 시공능력평가 순위 25위인 동부건설이 결국 세밑인 지난달 31일 법정관리(기업회생절차)를 신청하면서 동종업계에 위기감이 확산하고 있다. 특히 정부의 잇단 부동산시장 살리기 대책에도 건설업 수익이 계속 줄자 사업 전망에 대한 회의론까지 나오고 있다. 동시에 건설업 전반의 수익 감소는 근본적인구조적 문제로 정책적 개선이 필요하다는 의견도 제기되고 있다. 업계에 따르면 1일 현재 시공능력평가순위 100위 내 건설사 중 법정관리와 워크아웃(기업개선작업)에 들어간 건설사는 총 17곳이다. 최근 법정관리를 신청한 동부건설까지 포함하면 총 18개사가 부도 위기에 놓인 것이다. 업계는 동부건설에 이어 올해도 법정관리를 신청하는 건설사가 나올 것으로 전망하고 있다. 실제 유동성 위기에 처한 건설사는 여럿이다. 대한건설협회가 지난해 3분기 기준 124개 상장 건설사의 경영 실적을 분석한 결과, 자본 총액이 0 이하의 자본 잠식 상태 건설사는 10곳에 달하는 것으로 나타났다. 이들 건설사는 대부분 중견업체들로 사실상 부도 위기에 직면한 상태다. 건설업계의 경영 실적도 1년 전보다 더 나빠졌다. 지난해 3분기 124개 건설사의 영업이익율은 전년도 같은 기간보다 1.3%포인트 감소한 1.0%로 조사됐다. 한국은행이 최근 발표한 제조업 영업이익률 3.3%를 크게 밑도는 수준이다. 또 조사 대상의 43.5%인 54개사가 당기순손실을 기록했다. 평균 부채비율도 175.8%로 1년 만에 5.8%포인트나 증가했다. 대한건설협회 건설정보실 김영희 팀장은 건설업계의 경영 부진은 단순히 개별 기업의 부실 경영 때문이 아닌 갈수록 수익이 줄어드는 구조적인 문제가 주된 원인이라며 제값 주고 제대로 시공하기 등 구조적 문제를 개선해야 한다고 말했다. 눈길을 끄는 것은 몇 년새 바뀐 건설산업 환경이다. 지난해 주택사업이 활발한 건설사는 그나마 경영 실적이 개선된 반면 해외사업 비중이 큰 건설사는 적자를 본 곳이 많다. 몇 년 전까지만해도 주택사업

In [67]:
from konlpy.tag import Mecab
mecab = Mecab()

tokenized_docs = df['content'].apply(lambda x: mecab.nouns(x))
tokenized_docs = tokenized_docs.apply(lambda x: [item for item in x if len(item)>1])
tokenized_docs

0        [미년, 새해, 초부, 건설업, 칼바람, 시공, 능력, 평가, 순위, 동부건설, 결...
1        [중화학, 업계, 다사다난, 사고, 안전, 관리, 문제, 도마, 글로벌, 경기, 침...
2        [상반기, 기준, 금리, 추가, 하락, 초저, 금리, 기조, 은행, 만기, 정기예금...
3        [조직, 정비, 전자, 업계, 올해, 안정, 성장, 다지기, 주력, 삼성전자, 스마...
4        [지역, 단장, 동부, 광진, 나주호, 충북, 송정섭, 충남, 임장혁, 무등, 박희...
                               ...                        
34942    [대표, 이사, 내정, 한화, 글로벌, 부문, 김맹윤, 방산, 부문, 김승모, 한화...
34943    [흥국, 증권, 한화, 솔루션, 미국, 수소, 업체, 니콜라, 이슈, 주가, 반영,...
34944    [한화그룹, 한화, 솔루션, 전략, 부문, 김동관, 부사장, 사진, 사장, 승진, ...
34945    [한화그룹, 장단, 인사, 김승연, 한화그룹, 회장, 장남, 김동관, 한화, 솔루션...
34946    [한화그룹, 계열사, 장단, 인사, 단행, 왼쪽, 한화, 글로벌, 부문, 김맹윤, ...
Name: content, Length: 34947, dtype: object

In [65]:
tokenized_docs[9]

&#39;김승연 한화그룹 회장 그룹 변혁기 분야 기본 시너지 강조 김승연 회장 서울 중구 장교동 사옥 주요 임원 참석 가운데 신년 인사 한화그룹 그동안 환경 미래 투자 도전 내실 기반 대통 합의 기틀 시너지 확대 도약 원년 당부 회장 시작 마음 여러분 작년 연말 유수 방산 화학 회사 가족 변혁 발걸음 그룹 력사 자리 케미 생명 보험 인수 그룹 명운 역사 도전 방산 화학 부문 그룹 선대 회장 취임 당시 열정 사업 사명감 회사 일류 기업 통합 법인 출범 태양광 사업 세계 최고 경쟁력 확보 정상 궤도 금융 서비스 부문 목표 도전 주문 예전 그룹 성장 하이브리드 문화 중요 강조 인재 전쟁 시대 가족 임직원 천군만마 존재 가족 그룹 안정 정착 협력 멀리 정신 목표 강조 회장 위기 시대 초심 기본 원칙 충실 교훈 각자 혁신 업무 마인드 무장 역할 업계 최고 수준 수행 일류 경영 성과 자연 달성 사회 약자 현장 책임 기업 소명 한화 정신 재무장 바탕 사업 보국 창업 이념 계승 발전&#39;

# LDA

In [68]:
from gensim import corpora
from gensim.models import LdaModel, TfidfModel

id2word = corpora.Dictionary(tokenized_docs)
corpus_TDM = [id2word.doc2bow(doc) for doc in tokenized_docs]
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 100
lda = LdaModel(corpus=corpus_TFIDF,
                    id2word=id2word,
                    num_topics=n, 
                    random_state=100)

for t in lda.print_topics() :
  print(t)

(6, &#39;0.000*&quot;글래스고&quot; + 0.000*&quot;멜레&quot; + 0.000*&quot;화악&quot; + 0.000*&quot;드완&quot; + 0.000*&quot;묘회&quot; + 0.000*&quot;피사&quot; + 0.000*&quot;산타마리아&quot; + 0.000*&quot;파산법&quot; + 0.000*&quot;사민&quot; + 0.000*&quot;푯값&quot;&#39;)
(72, &#39;0.033*&quot;부문장&quot; + 0.028*&quot;파트&quot; + 0.018*&quot;단장&quot; + 0.013*&quot;비즈&quot; + 0.012*&quot;충청&quot; + 0.012*&quot;리서치&quot; + 0.011*&quot;연수&quot; + 0.009*&quot;분당&quot; + 0.008*&quot;아산&quot; + 0.008*&quot;지역&quot;&#39;)
(87, &#39;0.018*&quot;이스트&quot; + 0.016*&quot;공군&quot; + 0.007*&quot;이노&quot; + 0.005*&quot;푸틴&quot; + 0.002*&quot;음주&quot; + 0.002*&quot;권역&quot; + 0.000*&quot;하우스&quot; + 0.000*&quot;전셋값&quot; + 0.000*&quot;미사일&quot; + 0.000*&quot;가구&quot;&#39;)
(49, &#39;0.000*&quot;글래스고&quot; + 0.000*&quot;멜레&quot; + 0.000*&quot;화악&quot; + 0.000*&quot;드완&quot; + 0.000*&quot;묘회&quot; + 0.000*&quot;피사&quot; + 0.000*&quot;산타마리아&quot; + 0.000*&quot;파산법&quot; + 0.000*&quot;사민&quot; + 0.000*&quot;푯값&quot;&#39;)
(58, &#

In [76]:
lda.print_topics(100)

[(0,
  &#39;0.032*&quot;모델&quot; + 0.031*&quot;모듈&quot; + 0.030*&quot;장비&quot; + 0.026*&quot;연료전지&quot; + 0.026*&quot;성능&quot; + 0.025*&quot;탄소&quot; + 0.024*&quot;전기&quot; + 0.021*&quot;양산&quot; + 0.021*&quot;소재&quot; + 0.019*&quot;배터리&quot;&#39;),
 (1,
  &#39;0.009*&quot;달러&quot; + 0.006*&quot;중국&quot; + 0.006*&quot;데일리&quot; + 0.006*&quot;미국&quot; + 0.006*&quot;사고&quot; + 0.006*&quot;금융&quot; + 0.006*&quot;기업&quot; + 0.005*&quot;정부&quot; + 0.005*&quot;확산&quot; + 0.005*&quot;발생&quot;&#39;),
 (2,
  &#39;0.000*&quot;글래스고&quot; + 0.000*&quot;멜레&quot; + 0.000*&quot;화악&quot; + 0.000*&quot;드완&quot; + 0.000*&quot;묘회&quot; + 0.000*&quot;피사&quot; + 0.000*&quot;산타마리아&quot; + 0.000*&quot;파산법&quot; + 0.000*&quot;사민&quot; + 0.000*&quot;푯값&quot;&#39;),
 (3,
  &#39;0.037*&quot;유동&quot; + 0.034*&quot;유로&quot; + 0.028*&quot;밸류&quot; + 0.024*&quot;컨퍼런스&quot; + 0.023*&quot;레버리지&quot; + 0.014*&quot;셰프&quot; + 0.013*&quot;의류&quot; + 0.011*&quot;음식&quot; + 0.011*&quot;경신&quot; + 0.009*&quot;한중&quot;&#39;)

In [ ]:
# documents = df.content.tolist()
# documents

In [27]:
# # Mecab 토큰화
# from konlpy.tag import Mecab
# mecab = Mecab()

# data_word=[]
# for i in range(len(documents)):
#     data_word.append(mecab.nouns(documents[i]))

# data_word

In [71]:
data_word = tokenized_docs

In [72]:
import warnings
warnings.filterwarnings("ignore")

import os
from gensim import corpora
from gensim.models.wrappers import LdaMallet

id2word=corpora.Dictionary(data_word)
id2word.filter_extremes(no_below = 20) #20회 이하로 등장한 단어는 삭제
texts = data_word
corpus=[id2word.doc2bow(text) for text in texts]

os.environ['Mallet_HOME'] = 'C:\\Mallet'
mallet_path = 'C:\\Mallet\\bin\\mallet' 
ldamallet = LdaMallet(mallet_path, corpus=corpus, num_topics=100, id2word=id2word)

In [74]:
ldamallet.print_topics(100)

[(0,
  &#39;0.157*&quot;개발&quot; + 0.137*&quot;기술&quot; + 0.070*&quot;연구&quot; + 0.028*&quot;분야&quot; + 0.017*&quot;연구소&quot; + 0.017*&quot;한국&quot; + 0.014*&quot;세계&quot; + 0.013*&quot;국내&quot; + 0.013*&quot;확보&quot; + 0.012*&quot;핵심&quot;&#39;),
 (1,
  &#39;0.099*&quot;전문&quot; + 0.059*&quot;뱅크&quot; + 0.042*&quot;카카오&quot; + 0.036*&quot;인터넷&quot; + 0.027*&quot;참여&quot; + 0.023*&quot;케이&quot; + 0.021*&quot;컨소시엄&quot; + 0.016*&quot;토스&quot; + 0.016*&quot;은행&quot; + 0.015*&quot;예비&quot;&#39;),
 (2,
  &#39;0.033*&quot;모델&quot; + 0.022*&quot;최고&quot; + 0.022*&quot;엔진&quot; + 0.017*&quot;가격&quot; + 0.017*&quot;마력&quot; + 0.016*&quot;공개&quot; + 0.015*&quot;적용&quot; + 0.014*&quot;최대&quot; + 0.014*&quot;출력&quot; + 0.013*&quot;판매&quot;&#39;),
 (3,
  &#39;0.149*&quot;정부&quot; + 0.080*&quot;경제&quot; + 0.061*&quot;정책&quot; + 0.022*&quot;민간&quot; + 0.022*&quot;일자리&quot; + 0.021*&quot;국가&quot; + 0.020*&quot;발표&quot; + 0.017*&quot;예산&quot; + 0.017*&quot;투자&quot; + 0.016*&quot;재정&quot;&#39;),
 (4,
 